# VIII - Keras+TensorFlow Single GPU Training
In the previous notebook we have been using Microsoft's Cognitive Toolkit (CNTK) as our Deep Learning framework. Batch Shipyard though is not limited to CNTK and can be used with any Deep Learning framework. A very popular combination is TensorFlow and Keras. Keras is an easy to use high level API for TensorFlow, Theano and more recently CNTK, which makes creating artificial neural networks easy. In this notebook we will quickly demonstrate how to train a CNN using Keras on Batch Shipyard. 

* [Setup](#section1)
* [Define our model](#section2)
* [Configure and create pool](#section3)
* [Configure job](#section4)
* [Submit job](#section5)
* [Delete job](#section6)

<a id='section1'></a>

## Setup

Create a simple alias for Batch Shipyard

In [1]:
%alias shipyard SHIPYARD_CONFIGDIR=config python $HOME/batch-shipyard/shipyard.py %l

Check that everything is working

In [2]:
shipyard --version

shipyard.py, version 2.9.0rc1


Read in the account information we saved earlier

In [3]:
import json

def read_json(filename):
    with open(filename, 'r') as infile:
        return json.load(infile)
    
account_info = read_json('account_information.json')

STORAGE_ALIAS = account_info['STORAGE_ALIAS']

<a id='section2'></a>
## Define Our Model
The file below contains a simple CNN written in Keras. It will load the CIFAR 10 data and then train the model for a number of epochs and then evaludate it on the test set.

In [4]:
%%writefile cifar10_cnn.py
'''Train a simple deep CNN on the CIFAR10 small images dataset.
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

batch_size = 32
num_classes = 10
epochs = 20
data_augmentation = True

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))


Writing cifar10_cnn.py


<a id='section3'></a>
## Configure  and Create Pool
Here we will simply be using a pool with a single node just as a demonstration. Upon pool creation Batch Shipyard will pull our model file above and place into the blob container **input**.

In [5]:
IMAGE_NAME = "masalvar/keras" # Keras image

In [6]:
INPUT_CONTAINER = 'input'
UPLOAD_DIR = 'dist_upload'

!rm -rf $UPLOAD_DIR
!mkdir -p $UPLOAD_DIR
!mv cifar10_cnn.py $UPLOAD_DIR
!ls -alF $UPLOAD_DIR

total 12
drwxr-xr-x  2 nbuser nbuser 4096 Aug 11 13:24 ./
drwx------ 20 nbuser nbuser 4096 Aug 11 13:24 ../
-rw-r--r--  1 nbuser nbuser 3445 Aug 11 13:24 cifar10_cnn.py


In [7]:
config = {
    "batch_shipyard": {
        "storage_account_settings": STORAGE_ALIAS
    },
    "global_resources": {
        "docker_images": [
            IMAGE_NAME
        ],
        "files": [
            {
                "source": {
                    "path": UPLOAD_DIR
                },
                "destination": {
                    "storage_account_settings": STORAGE_ALIAS,
                    "data_transfer": {
                        "container": INPUT_CONTAINER
                    }
                }
            }
        ]
    }
}

In [8]:
POOL_ID = 'gpupool-keras'

pool = {
    "pool_specification": {
        "id": POOL_ID,
        "vm_configuration": {
            "platform_image": {
                "publisher": "Canonical",
                "offer": "UbuntuServer",
                "sku": "16.04-LTS"
            },
        },
        "vm_size": "STANDARD_NC6",
        "vm_count": {
            "dedicated": 1
        },
        "ssh": {
            "username": "docker"
        },
        "reboot_on_start_task_failed": False,
        "block_until_all_global_resources_loaded": True,
        "input_data": {
            "azure_storage": [
                {
                    "storage_account_settings": STORAGE_ALIAS,
                    "container": INPUT_CONTAINER,
                    "destination": "$AZ_BATCH_NODE_SHARED_DIR/code"
                }
            ]
        },
        "transfer_files_on_pool_creation": True,
    }
}

In [9]:
import json
import os

def write_json_to_file(json_dict, filename):
    """ Simple function to write JSON dictionaries to files
    """
    with open(filename, 'w') as outfile:
        json.dump(json_dict, outfile)

In [10]:
write_json_to_file(config, os.path.join('config', 'config.json'))
write_json_to_file(pool, os.path.join('config', 'pool.json'))
print(json.dumps(config, indent=4, sort_keys=True))
print(json.dumps(pool, indent=4, sort_keys=True))

{
    "batch_shipyard": {
        "storage_account_settings": "mystorageaccount"
    }, 
    "global_resources": {
        "docker_images": [
            "masalvar/keras"
        ], 
        "files": [
            {
                "destination": {
                    "data_transfer": {
                        "container": "input"
                    }, 
                    "storage_account_settings": "mystorageaccount"
                }, 
                "source": {
                    "path": "dist_upload"
                }
            }
        ]
    }
}
{
    "pool_specification": {
        "block_until_all_global_resources_loaded": true, 
        "id": "gpupool-keras", 
        "input_data": {
            "azure_storage": [
                {
                    "container": "input", 
                    "destination": "$AZ_BATCH_NODE_SHARED_DIR/code", 
                    "storage_account_settings": "mystorageaccount"
                }
            ]
        }, 
        "reboot_on_

Allocate the pool, please be patient for this step.

In [11]:
shipyard pool add -y

2017-08-11 13:24:37,839 INFO - creating container: shipyardremotefs
2017-08-11 13:24:38,053 INFO - creating table: shipyarddht
2017-08-11 13:24:38,250 INFO - creating table: shipyardtorrentinfo
2017-08-11 13:24:38,292 INFO - creating table: shipyardgr
2017-08-11 13:24:38,335 INFO - creating table: shipyardregistry
2017-08-11 13:24:38,375 INFO - creating container: shipyardrf-batch71d77646ba-gpupool-keras
2017-08-11 13:24:38,468 INFO - creating table: shipyardimages
2017-08-11 13:24:38,508 INFO - creating container: shipyardtor-batch71d77646ba-gpupool-keras
2017-08-11 13:24:38,566 INFO - creating container: shipyardgr-batch71d77646ba-gpupool-keras
2017-08-11 13:24:38,617 DEBUG - clearing table (pk=batch71d77646ba$gpupool-keras): shipyarddht
2017-08-11 13:24:38,658 DEBUG - clearing table (pk=batch71d77646ba$gpupool-keras): shipyardtorrentinfo
2017-08-11 13:24:38,699 DEBUG - clearing table (pk=batch71d77646ba$gpupool-keras): shipyardgr
2017-08-11 13:24:38,739 DEBUG - clearing table (pk=ba

<a id='section4'></a>
## Configure Job
As before the dictionary below defines the job we will execute. 

In [13]:
TASK_ID = 'run_cifar10' # This should be changed per task

JOB_ID = 'keras-training-job'

COMMAND = 'bash -c "python -u $AZ_BATCH_NODE_SHARED_DIR/code/cifar10_cnn.py"'

jobs = {
    "job_specifications": [
        {
            "id": JOB_ID,
            "tasks": [
                {
                    "id": TASK_ID,
                    "image": IMAGE_NAME,
                    "remove_container_after_exit": True,
                    "command": COMMAND,
                    "gpu": True,
                }
            ],
        }
    ]
}

Write the jobs configuration to the `jobs.json` file:

In [14]:
write_json_to_file(jobs, os.path.join('config', 'jobs.json'))
print(json.dumps(jobs, indent=4, sort_keys=True))

{
    "job_specifications": [
        {
            "id": "keras-training-job", 
            "tasks": [
                {
                    "command": "bash -c \"python -u $AZ_BATCH_NODE_SHARED_DIR/code/cifar10_cnn.py\"", 
                    "gpu": true, 
                    "id": "run_cifar10", 
                    "image": "masalvar/keras", 
                    "remove_container_after_exit": true
                }
            ]
        }
    ]
}


<a id='section5'></a>

## Submit Job
Check that everything is ok with our pool before we submit our jobs.

In [15]:
shipyard pool listnodes

2017-08-11 13:47:25,600 INFO - compute nodes for pool gpupool-keras
* node id: tvm-3257026573_1-20170811t132558z
  * state: ComputeNodeState.idle
  * scheduling state: SchedulingState.enabled
  * no errors
  * start task:
    * exit code: 0
    * started: 2017-08-11 13:27:04.481808+00:00
    * completed: 2017-08-11 13:31:07.147640+00:00
    * duration: 0:04:02.665832
  * vm size: standard_nc6
  * dedicated: True
  * ip address: 10.0.0.4
  * running tasks: 0
  * total tasks run: 0
  * total tasks succeeded: 0


Now that we have confirmed everything is working we can execute our job using the command below. The tail switch at the end will stream stdout from the node.

In [16]:
shipyard jobs add --tail stdout.txt

2017-08-11 13:47:29,491 INFO - Adding job keras-training-job to pool gpupool-keras
2017-08-11 13:47:30,012 INFO - uploading file /tmp/tmpgNLR1W as u'shipyardtaskrf-keras-training-job/run_cifar10.shipyard.envlist'
2017-08-11 13:47:30,060 DEBUG - submitting 1 tasks (0 -> 0) to job keras-training-job
2017-08-11 13:47:30,289 INFO - submitted all 1 tasks to job keras-training-job
2017-08-11 13:47:30,289 DEBUG - attempting to stream file stdout.txt from job=keras-training-job task=run_cifar10
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
UUsing real-time data augmentation.
Epoch 1/20
1562/1562 [==============================] - 22s - loss: 1.8966 - acc: 0.3029 - val_loss: 1.5810 - val_acc: 0.4318====>.........................] - ETA: 23s - loss: 2.1900 - acc: 0.1733

If something goes wrong you can run the following command to get the stderr output from the job.

In [17]:
shipyard data stream --filespec $JOB_ID,$TASK_ID,stderr.txt

2017-08-11 13:55:33,502 DEBUG - attempting to stream file stderr.txt from job=keras-training-job task=run_cifar10
Using TensorFlow backend.
2017-08-11 13:47:56.546740: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2017-08-11 13:47:56.546800: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2017-08-11 13:47:56.546819: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2017-08-11 13:47:56.546832: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and co

<a id='section6'></a>

## Delete job

To delete the job use the command below. Just be aware that this will get rid of all the files created by the job and tasks.

In [ ]:
shipyard jobs del -y --termtasks --wait

2017-08-11 13:55:42,739 DEBUG - Skipping termination of completed task run_cifar10 on job keras-training-job
2017-08-11 13:55:42,949 INFO - deleting job: keras-training-job
2017-08-11 13:55:43,110 DEBUG - waiting for job keras-training-job to delete
2017-08-11 13:56:14,225 INFO - job keras-training-job does not exist


In [ ]:
shipyard pool del -y --wait

2017-08-11 13:56:15,958 INFO - Deleting pool: gpupool-keras
2017-08-11 13:56:16,253 DEBUG - clearing table (pk=batch71d77646ba$gpupool-keras): shipyarddht
2017-08-11 13:56:16,479 DEBUG - clearing table (pk=batch71d77646ba$gpupool-keras): shipyardtorrentinfo
2017-08-11 13:56:16,521 DEBUG - clearing table (pk=batch71d77646ba$gpupool-keras): shipyardgr
2017-08-11 13:56:16,604 DEBUG - clearing table (pk=batch71d77646ba$gpupool-keras): shipyardregistry
2017-08-11 13:56:16,688 DEBUG - clearing table (pk=batch71d77646ba$gpupool-keras): shipyardimages
2017-08-11 13:56:16,780 DEBUG - clearing table (pk=batch71d77646ba$gpupool-keras): shipyardperf
2017-08-11 13:56:16,821 DEBUG - deleting queue: shipyardgr-batch71d77646ba-gpupool-keras
2017-08-11 13:56:17,015 DEBUG - deleting container: shipyardrf-batch71d77646ba-gpupool-keras
2017-08-11 13:56:17,251 DEBUG - deleting container: shipyardtor-batch71d77646ba-gpupool-keras
2017-08-11 13:56:17,300 DEBUG - deleting container: shipyardgr-batch71d77646ba